In [1]:
import os
import pandas as pd
import datetime


In [69]:
folder_path = 'C:/Users/User/Desktop/VSCpython/senior_prj/source/'
files = os.listdir(folder_path)
new_path = 'C:/Users/User/Desktop/VSCpython/senior_prj/load_newmeter/'
old_path = 'C:/Users/User/Desktop/VSCpython/senior_prj/load_oldmeter/'
column = ['datetime','timediff(ms)', 'Ia(A)',
                          'Ib(A)',
                          'Ic(A)',
                          'In(A)',
                          'Iavg(A)',
                          'Va(V)',
                          'Vb(V)',
                          'Vc(V)',
                          'Vavg(V)',
                          'Pa(kW)',
                          'Pb(kW)',
                          'Pc(kW)',
                          'Ptot(kW)',
                          'VARa(kVAR)',
                          'VARb(kVAR)',
                          'VARc(kVAR)',
                          'VARtot(kVAR)',
                          'VAa(V)',
                          'VAb(V)',
                          'VAc(V)',
                          'VAtot(V)',
                          'PFa',
                          'PFb',
                          'PFc',
                          'PFtot',
                          'DFa',
                          'DFb',
                          'DFc',
                          'DFtot']
error_list = []
for file in files:
    if file.lower().startswith('load_gw') and file.lower().endswith('.csv'):
        df = pd.read_csv(folder_path  + file)
        print('importing' + folder_path  + file )
        df.columns = ['DateTime', 'Energy(Wh)']
        
    elif file.lower().startswith('load507') and file.lower().endswith('.csv'):
        try :
            df = pd.read_csv(folder_path  + file,header = None ).dropna(axis = 0)
            df1 = pd.DataFrame(columns = column)
            print('importing' + folder_path  + file )
        
            start_value = df.iloc[0,0] 
            if len(file[4:-4]) == 3 :
                date = '2023' + file[-6:-4] + '0' + file[4]
            elif len(file[4:-4]) == 4 :
                date = '2023' + file[-6:-4] + file[4:6]
            
            start_date = datetime.datetime(int(date[:4]),int(date[4:6]),int(date[6:8]))
            
            if len(df.columns) == 30 :
                
                df.iloc[:,0] = (df.iloc[:,0] - start_value)*1000
                df1[column[1:]] = df
                df1['datetime'] = df1['timediff(ms)'].apply(lambda x: start_date + datetime.timedelta(milliseconds = x))
                df1.to_excel(new_path + 'load_newmeter_' + date + '.xlsx',index = False)
                print('export finished')
            if len(df.columns) == 12 :
                df.iloc[:,0] = (df.iloc[:,0] - start_value)*1000
                
                
                
                df1[['timediff(ms)','Ia(A)','Ib(A)','Ic(A)','Pa(kW)','Pb(kW)','Pc(kW)','Ptot(kW)','VAa(V)','VAb(V)','VAc(V)','VAtot(V)']] = df
                df1[['Pa(kW)','Pb(kW)','Pc(kW)','Ptot(kW)']] = df1[['Pa(kW)','Pb(kW)','Pc(kW)','Ptot(kW)']]/1000
                df1['datetime'] = df1['timediff(ms)'].apply(lambda x: start_date + datetime.timedelta(milliseconds = x))
                df1.to_excel(old_path + 'load_oldmeter_' + date + '.xlsx',index = False)
                print('export finished')
        except Exception as e :
                error_list.append( [file,str(e)])
if error_list != [] :
    error_list = pd.DataFrame(error_list)
    error_list.to_excel('error_log.xlsx',index = False)

            


importingC:/Users/User/Desktop/VSCpython/senior_prj/source/load507.csv
export finished


In [2]:
column = ['datetime','timediff(ms)', 'Ia(A)',
                          'Ib(A)',
                          'Ic(A)',
                          'In(A)',
                          'Iavg(A)',
                          'Va(V)',
                          'Vb(V)',
                          'Vc(V)',
                          'Vavg(V)',
                          'Pa(kW)',
                          'Pb(kW)',
                          'Pc(kW)',
                          'Ptot(kW)',
                          'VARa(kVAR)',
                          'VARb(kVAR)',
                          'VARc(kVAR)',
                          'VARtot(kVAR)',
                          'VAa(V)',
                          'VAb(V)',
                          'VAc(V)',
                          'VAtot(V)',
                          'PFa',
                          'PFb',
                          'PFc',
                          'PFtot',
                          'DFa',
                          'DFb',
                          'DFc',
                          'DFtot']

In [22]:
folder_path = 'C:/Users/User/Desktop/VSCpython/senior_prj/load_reformat/'
files = os.listdir(folder_path) 
output_path = 'C:/Users/User/Desktop/VSCpython/senior_prj/load_downsampling/'
for file in files :
    if (file.lower().startswith('load_new')  or  file.lower().startswith('load_old') ) and file.lower().endswith('.xlsx') :
        df = pd.read_excel(folder_path  + file )
        print('importing' + folder_path  + file )
        start_date = file[-13:-5]
        start_date = datetime.datetime(int(start_date[:4]),int(start_date[4:6]),int(start_date[6:8]))
        df['period'] = 0
        df['period'][:-1] = pd.Series( df['timediff(ms)']).diff(periods= 1 )[1:]
        df['datetime'] = start_date + (df['timediff(ms)']//900000)*datetime.timedelta(minutes = 15)
        print('a')
        for e in column[2:32] :
            df[e] = df[e] * df['period']
        df_return = df.groupby('datetime').sum(column[2:32])
        for e in column[2:32] :
            df_return[e] = df_return[e]/df_return['period']
        print('b')
        df_return.drop(['timediff(ms)','period'],axis = 1,inplace = True)
        df_return.to_excel(output_path + file )
        print('export finished')


importingC:/Users/User/Desktop/VSCpython/senior_prj/load_reformat/load_newmeter_20230508.xlsx
a
b


C:\Users\User\AppData\Local\Temp\ipykernel_16880\853392660.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['period'][:-1] = pd.Series( df['timediff(ms)']).diff(periods= 1 )[1:]


export finished
importingC:/Users/User/Desktop/VSCpython/senior_prj/load_reformat/load_newmeter_20230509.xlsx
a
b


C:\Users\User\AppData\Local\Temp\ipykernel_16880\853392660.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['period'][:-1] = pd.Series( df['timediff(ms)']).diff(periods= 1 )[1:]


export finished
importingC:/Users/User/Desktop/VSCpython/senior_prj/load_reformat/load_newmeter_20230511.xlsx
a
b


...

export finished
importingC:/Users/User/Desktop/VSCpython/senior_prj/load_reformat/load_oldmeter_20230504.xlsx
a
b


C:\Users\User\AppData\Local\Temp\ipykernel_16880\853392660.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['period'][:-1] = pd.Series( df['timediff(ms)']).diff(periods= 1 )[1:]


export finished
importingC:/Users/User/Desktop/VSCpython/senior_prj/load_reformat/load_oldmeter_20230505.xlsx
a
b


C:\Users\User\AppData\Local\Temp\ipykernel_16880\853392660.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['period'][:-1] = pd.Series( df['timediff(ms)']).diff(periods= 1 )[1:]


export finished


In [17]:
df_return

,timediff(ms),Ia(A),Ib(A),Ic(A),In(A),Iavg(A),Va(V),Vb(V),Vc(V),Vavg(V),...,VAtot(V),PFa,PFb,PFc,PFtot,DFa,DFb,DFc,DFtot,period
datetime,,,,,,,,,,,,,,,,,,,,,
2023-07-10 00:00:00,1.000930e+09,1.142108,0.242802,2.034238,1.891887,1.140508,228.018904,229.772996,227.478812,228.423552,...,0.778950,0.846829,0.322154,0.861057,0.815119,0.879809,0.130042,0.872779,0.627139,900130.0
2023-07-10 00:15:00,3.002166e+09,1.108847,0.241079,1.945913,1.813039,1.099916,226.349038,228.121286,225.812516,226.760944,...,0.746248,0.845917,0.320843,0.854869,0.814765,0.878945,0.119645,0.867235,0.622242,900000.0
2023-07-10 00:30:00,5.003569e+09,1.309910,0.240099,1.995888,1.883936,1.182388,226.159669,227.916532,225.599615,226.558616,...,0.800988,0.859374,0.319375,0.858221,0.826001,0.889688,0.122540,0.870419,0.627644,900250.0
2023-07-10 00:45:00,7.002628e+09,1.107117,0.240000,2.019150,1.894961,1.121843,226.333355,228.090192,225.818169,226.747380,...,0.760358,0.846085,0.319580,0.861713,0.820024,0.879170,0.126197,0.875747,0.626036,899750.0
2023-07-10 01:00:00,9.006963e+09,1.155965,0.240000,1.974818,1.849258,1.123875,226.774214,228.378812,226.170298,227.107708,...,0.762365,0.848900,0.319676,0.856758,0.818769,0.880201,0.117505,0.867699,0.621890,899870.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-10 22:45:00,1.825691e+11,4.377084,2.091911,6.072533,6.148567,4.180417,226.869159,228.622966,226.367315,227.286418,...,2.846087,0.954474,0.653879,0.986098,0.900425,0.977501,0.664454,0.985512,0.876806,900250.0
2023-07-10 23:00:00,1.846481e+11,5.090017,2.089278,5.888192,6.102333,4.355792,226.766525,228.471186,226.288236,227.175322,...,2.962642,0.962081,0.655089,0.981503,0.904294,0.981953,0.665392,0.985433,0.879036,900000.0
2023-07-10 23:15:00,1.865607e+11,4.150461,2.102381,5.832520,4.204676,4.028191,227.286713,229.061364,226.795026,227.714351,...,2.743843,0.945365,0.651989,0.977383,0.876963,0.969675,0.662876,0.979997,0.871826,899750.0


In [66]:
df_return.to_frame()

AttributeError: 'DataFrame' object has no attribute 'to_frame'

C:\Users\User\AppData\Local\Temp\ipykernel_2600\1231091239.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['period'][:-1] = pd.Series( df['timediff(ms)']).diff(periods= 1 )[1:]


In [42]:
df.groupby('group').sum(column[2:])/900000

,timediff(ms),Ia(A),Ib(A),Ic(A),In(A),Iavg(A),Va(V),Vb(V),Vc(V),Vavg(V),...,VAtot(V),PFa,PFb,PFc,PFtot,DFa,DFb,DFc,DFtot,period
group,,,,,,,,,,,,,,,,,,,,,
0.0,675.463233,1.512821,2.066524,1.589254,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000767
1.0,2023.460022,1.525083,2.192855,1.663594,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.999233
2.0,3370.117111,1.540576,2.239093,1.679201,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000622
3.0,4725.329333,1.540495,1.822306,1.597928,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000
4.0,6075.320722,1.529372,1.543957,1.431489,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91.0,123524.519467,0.333451,1.875337,1.011844,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000
92.0,124874.515200,0.351867,1.920437,1.092241,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000
93.0,126224.511067,0.329201,2.268657,1.144920,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000


In [41]:
df

,datetime,timediff(ms),Ia(A),Ib(A),Ic(A),In(A),Iavg(A),Va(V),Vb(V),Vc(V),...,PFa,PFb,PFc,PFtot,DFa,DFb,DFc,DFtot,period,group
0,2023-03-01 00:00:00.000,0.0,936.2,1190.4,954.8,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,620.0,0.0
1,2023-03-01 00:00:00.620,620.0,951.3,1209.6,970.2,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,630.0,0.0
2,2023-03-01 00:00:01.250,1250.0,1132.5,1440.0,1155.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,750.0,0.0
3,2023-03-01 00:00:02.000,2000.0,1041.9,1324.8,1076.4,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,690.0,0.0
4,2023-03-01 00:00:02.690,2690.0,936.2,1190.4,967.2,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,620.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129536,2023-03-01 23:59:40.000,86380000.0,213.9,1221.3,669.3,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,690.0,95.0
129537,2023-03-01 23:59:40.690,86380690.0,192.2,1097.4,601.4,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,620.0,95.0
129538,2023-03-01 23:59:41.310,86381310.0,213.9,1221.3,669.3,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,690.0,95.0
129539,2023-03-01 23:59:42.000,86382000.0,192.2,1097.4,601.4,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,620.0,95.0


In [26]:
df1

,datetime,timediff(ms),Ia(A),Ib(A),Ic(A),In(A),Iavg(A),Va(V),Vb(V),Vc(V),...,VAc(V),VAtot(V),PFa,PFb,PFc,PFtot,DFa,DFb,DFc,DFtot
0,2023-03-10 00:00:00.000,0.0,10.71,11.96,8.00,NaN,NaN,NaN,NaN,NaN,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-03-10 00:00:00.620,620.0,10.71,11.97,8.00,NaN,NaN,NaN,NaN,NaN,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-03-10 00:00:01.250,1250.0,10.71,11.97,8.00,NaN,NaN,NaN,NaN,NaN,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-03-10 00:00:01.870,1870.0,10.72,11.98,8.01,NaN,NaN,NaN,NaN,NaN,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-03-10 00:00:02.500,2500.0,10.72,11.98,8.01,NaN,NaN,NaN,NaN,NaN,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129555,2023-03-10 23:59:52.750,86392750.0,0.33,1.78,0.97,NaN,NaN,NaN,NaN,NaN,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
129556,2023-03-10 23:59:53.500,86393500.0,0.33,1.78,0.97,NaN,NaN,NaN,NaN,NaN,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
129557,2023-03-10 23:59:54.120,86394120.0,0.33,1.78,0.97,NaN,NaN,NaN,NaN,NaN,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
129558,2023-03-10 23:59:54.750,86394750.0,0.33,1.78,0.97,NaN,NaN,NaN,NaN,NaN,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
date

'20230310'

In [21]:
start_date = datetime.datetime(int(date[:4]),int(date[4:6]),int(date[6:8])) + datetime.timedelta(milliseconds = 100)
start_date

datetime.datetime(2023, 3, 10, 0, 0, 0, 100000)

In [ ]:
time(s), Ia, Ib, Ic, Pa, Pb, Pc, Ptot, VAa, VAb, VAc, VAtot

In [30]:
a = input().split(', ')
b = [e for e in a]
b


['time(s)',
 'Ia',
 'Ib',
 'Ic',
 'In',
 'Iavg',
 'Va',
 'Vb',
 'Vc',
 'Vavg',
 'Pa',
 'Pb',
 'Pc',
 'Ptot',
 'VARa',
 'VARb',
 'VARc',
 'VARtot',
 'VAa',
 'VAb',
 'VAc',
 'VAtot',
 'PFa',
 'PFb',
 'PFc',
 'PFtot',
 'DFa',
 'DFb',
 'DFc',
 'DFtot']